In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
vp1 = [136.6, 275.3, 560.0]

json_data = [[ 0.97606232, -0.00642975,  0.21739599], 
         [-0.21749105, -0.02885559,  0.97563569],
         [0.        , 0.99956291, 0.02956327]]

npz_data = [[ 0.97606232, -0.00642975,  0.21739599],
             [-0.21749105, -0.02885559,  0.97563569],
             [ 0.      ,    0.99956291,  0.02956327]]

In [16]:
def dot_product(vec1, vec2):
    result_dot = np.dot(vec1, vec2)
    return result_dot

def get_degree(vec1, vec2):
    vp_norm = np.linalg.norm(vec1)
    json_norm = np.linalg.norm(vec2)
    vp_json = dot_product(vec1, vec2)/vp_norm/json_norm

    degree = np.arccos(vp_json)
    return degree*180/np.pi

In [17]:
# degree 34.27
data1 = [417,207]
data2 = [424,206]

get_degree(data1, data2)

0.48717861916441

In [18]:
data1 = [417,207, 2.1875*256]
data2 = [424,206, 2.1875*256]

get_degree(data1, data2)

0.4705056486076437

In [19]:
gt = [0.28681979, 0.0835354,  0.9543355 ] 
pred = [417.3, 207.0, 2.1875*256]
get_degree(gt, pred)

22.818112170952315

### 바로 확인해보기

In [21]:
data1 = [231, 246, 2.1875*256]
data2 = [284, 265, 2.1875*256]

get_degree(data1, data2)

4.207699855737857

In [ ]:
data1 = [417, 207, 2.1875*256]
data2 = [424, 206, 2.1875*256]

get_degree(data1, data2)

In [22]:
data1 = [493, -114, 2.1875*256]
data2 = [256, -588, 2.1875*256]

get_degree(data1, data2)

37.93177521900293

In [26]:
data1 = [136-256, 275-256, 2.1875*256]
data2 = [131-256, 272-256, 2.1875*256]

get_degree(data1, data2)

0.5743044974798942

In [25]:
data1 = [-0.21749105, -0.02885559 , 0.97563569]
data2 = [136.6-256, 275.3-256, 2.1875*256]

get_degree(data1, data2)

3.6231567663216917

In [ ]:
data1 = [417, 207, 2.1875*256]
data2 = [424, 206, 2.1875*256]

get_degree(data1, data2)

In [ ]:
data1 = [417, 207, 2.1875*256]
data2 = [424, 206, 2.1875*256]

get_degree(data1, data2)

### 230602 hfov

In [4]:
# horizontal field of view angle of camera (unit: degree), logitech c930e
hfov = 82.1 # etridataset

hfov = 49.134342641202629     # SU3 dataset

hfov = 90 #  flickr dataset

In [14]:
def rad2deg(radi):
    return radi*180/np.pi

def deg2rad(deg):
    return np.pi*deg/180
#
# hfov_deg = rad2deg(np.arctan(img_w/2/f)*2)
hfov = 82.1

img_w = 1280
f = img_w/2/np.arctan(deg2rad(hfov/2))

In [15]:
f

1029.4582899424072

In [12]:
rad2deg(np.pi)

180.0

In [13]:
deg2rad(180)

3.141592653589793

In [ ]:
구조화
일반화

개선

### earlier

In [22]:
get_degree(vp1, json_data[0])

66.63824783844457

In [24]:
for j in json_data:
    print(get_degree(vp1, j))

66.63824783844457
37.216420031549276
62.82555217195277


In [25]:
for j in npz_data:
    print(get_degree(vp1, j))

66.63824783844457
37.216420031549276
62.82555217195277


In [26]:
for i,j in zip(json_data, npz_data):
    print(get_degree(i,j))

0.0
0.0
0.0


In [31]:
vp2 = [231, 246, 2.1875 * 256]
gt = [([-0.05008141,  0.01662795,  0.99860671]), 
      ([9.98745139e-01, 8.33797578e-04, 5.00744691e-02]), 
      ([-0.        , -0.9998614 ,  0.01664884])]

In [32]:
get_degree(vp2, gt)

array([ 78.06857845, 119.39737852,  77.12587792])

### 다시
보니까 몇 가지 문제가 있는데,
1. 일단 roadtheta 카메라의 focal length는 2.1875 * 256이 아니다.
 - focal length는 이미지 해상도에 따라 바뀌기 때문에 시야각으로 표현하는게 더 좋다.
 - roadtheta 카메라의 수평 시야각은 82.1도이다.
 - 이를 focal length로 변환하면 f = (img_w/2) / tan(deg2rad(82,1)/2) 가 된다.
2. vp랑 gt랑 각도 구할 때, vp를 카메라좌표로 변환한 후에 각도를 계산해야 한다.
 - vp = [231 - cx, 246 - cy, f]
 - gt도 카메라 좌표계로 변경해 줘야 한다 (gt_camera)
 - gt_camera = [gt(1) -gt(2) gt(3)]
3. gt의 z좌표가 음수일 때에는 gt의 부호를 바꿔줘야 한다
 - if gt(3)<0, gt = -gt; end (cvpr 논문에 있는 것처럼)
 - 부호를 바꾸지 않으면 gt와 vp가 같은 방향인 경우 사잇각이 180도가 나오게 된다.
 - ray의 진행방향이 반대가 되기 때문

내가 계산해 보니 22번 영상의 사잇각은 2.2212 degree 가 나온다.

In [37]:
# roadtheta 수평시야각 82.1도 => 어떻게 계산.. su3  49.13으로 설정
def deg2rad(x):
    return x*180/np.pi

img_w = 512

f = (img_w/2)/np.tan(deg2rad(82.1)/2)
print(f"new focal length : {f}")

new focal length : -143.07748088121676


In [66]:
# vp를 카메라좌표로 변환하여 각도계산해야함(cx, cy...)
def img2camera(point,cx=256,cy=256, f= 2.1875 * 256):
    new_vp = [point[0]-cx, point[1]-cy, f]
    return new_vp

def img2camera_gt(point,cx=256,cy=256):
    if point[2]<0:
        point[2] = -point[2]
    new_gt = [point[0],-point[1],point[2]]
    return new_gt

In [74]:
vp2 = [231, 246, 2.1875 * 256]
gt = [([-0.05008141,  0.01662795,  0.99860671]), 
      ([9.98745139e-01, 8.33797578e-04, 5.00744691e-02]), 
      ([-0.        , -0.9998614 ,  0.01664884])]

for j in gt:
    new_vp2 = img2camera(vp2,256,256)
    new_gt = img2camera_gt(j)
    print(f"vp : {new_vp2}, gt : {new_gt}")
    print(get_degree(new_vp2, new_gt))

vp : [-25, -10, 560.0], gt : [-0.05008141, -0.01662795, 0.99860671]
0.32238182693032247
vp : [-25, -10, 560.0], gt : [0.998745139, -0.000833797578, 0.0500744691]
89.68509094111951
vp : [-25, -10, 560.0], gt : [-0.0, 0.9998614, 0.01664884]
90.06900913986972


In [78]:
vp1 = [136, 275, 2.1875 * 256]

json_data = [[ 0.97606232, -0.00642975,  0.21739599], 
         [-0.21749105, -0.02885559,  0.97563569],
         [0.        , 0.99956291, 0.02956327]]

npz_data = [[ 0.97606232, -0.00642975,  0.21739599],
             [-0.21749105, -0.02885559,  0.97563569],
             [ 0.      ,    0.99956291,  0.02956327]]

for j in json_data:
    new_vp1 = img2camera(vp1,256,256)
    new_gt = img2camera_gt(j)
    print(f"vp : {new_vp1}, gt : {new_gt}")
    print(get_degree(new_vp1, new_gt))

vp : [-120, 19, 560.0], gt : [0.97606232, 0.00642975, 0.21739599]
89.52640844950288
vp : [-120, 19, 560.0], gt : [-0.21749105, 0.02885559, 0.97563569]
0.5325704741312551
vp : [-120, 19, 560.0], gt : [0.0, -0.99956291, 0.02956327]
90.24359732801533


## Test

In [10]:
# 기존 gt 값들은 정규화 평면에 있는 것임
gt_norm = np.linalg.norm(json_data[0])
gt_norm

1.0000000053384626

In [15]:
# 정규화 평면에 있는 것을 이미지 평면으로 옮기게 되면
gt_image_plane = [j*256 for j in json_data[0]]
gt_image_plane

[249.87195392, -1.646016, 55.65337344]